In [1]:
from ocpmodels.datasets import LmdbDataset
import lmdb
from ase import Atoms

In [2]:
slab_data_objs=LmdbDataset({"src": "datasetss/slabs.lmdb"})

In [7]:
atom_idx=0 
demo=slab_data_objs[atom_idx]
demo

Data(pos=[104, 3], cell=[1, 3, 3], atomic_numbers=[104], natoms=104, tags=[104], edge_index=[2, 4927], cell_offsets=[4927, 3], y=-869.1991577148438, force=[104, 3], fixed=[104], slab_formula='O72Ti32', unrelax_energy=-864.7914428710938, bulk_energy=-226.61505634, miller=[3], bulk_formula='O16Ti8')

# Function for getting surface energy from a Data object

In [19]:
from pymatgen.analysis.surface_analysis import SlabEntry
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.io.ase import AseAtomsAdaptor

def get_surface_energy(dat):
    bulk_entry = ComputedEntry(dat.bulk_formula, dat.bulk_energy)
    gas_entry = ComputedEntry('O2', 2*-7.204) # the ref energy for O in OC20
    
    atoms=Atoms(dat.atomic_numbers,
                positions=dat.pos,
                tags=dat.tags,
                cell=dat.cell.squeeze(), pbc=True)
    
    slabentry = SlabEntry(AseAtomsAdaptor.get_structure(atoms), dat.y, dat.miller)
    return slabentry.surface_energy(bulk_entry, [gas_entry])

In [18]:
from pymatgen.core.composition import Composition
for dat in slab_data_objs:
    print(Composition(dat.slab_formula).reduced_formula, 
          Composition(dat.bulk_formula).reduced_formula, 
          dat.miller, get_surface_energy(dat))

Ti4O9 TiO2 (1, 1, 1) 0.486508823034301 - 0.0410153310548295*delu_O
Ti9O20 TiO2 (1, 1, 1) 0.450274798852927 - 0.0410153310548295*delu_O
Ti8O17 TiO2 (1, 1, 1) 0.404413278395938 - 0.0205076655274147*delu_O
Ti9O19 TiO2 (1, 1, 1) 0.416724743705242 - 0.0205076655274147*delu_O
TiO2 TiO2 (1, 1, 1) 0.38411800535389684
TiO2 TiO2 (1, 1, 1) 0.3677720348620239
TiO2 TiO2 (1, 1, 1) 0.37070036027104436
TiO2 TiO2 (1, 1, 1) 0.3745788867635387
Ti10O19 TiO2 (1, 1, 1) 0.0205076655274147*delu_O + 0.445462215747588
Ti9O17 TiO2 (1, 1, 1) 0.0205076655274147*delu_O + 0.400585568918097
Ti5O9 TiO2 (1, 1, 1) 0.0410153310548295*delu_O + 0.472357140401416
Ti9O16 TiO2 (1, 1, 1) 0.0410153310548295*delu_O + 0.444017802955972
Ti4O9 TiO2 (1, 1, 0) 0.546207448328953 - 0.0489107017539134*delu_O
Ti8O17 TiO2 (1, 1, 0) 0.449417328042306 - 0.0244553508769567*delu_O
Ti8O17 TiO2 (1, 1, 0) 0.44118767856354 - 0.0244553508769567*delu_O
Ti9O19 TiO2 (1, 1, 0) 0.492222603655402 - 0.0244553508769567*delu_O
TiO2 TiO2 (1, 1, 0) 0.4229742